In [1]:
# --- Notebook: 2_model_training.ipynb ---

# --- 1. Import Libraries ---
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
    
# Core ML framework import (still needed for some direct calls like preprocess_input)
import tensorflow as tf

# Import functions from your custom src modules
from src.data_processing import get_image_data_generators, compute_class_weights
from src.model import build_transfer_model, compile_model, unfreeze_and_recompile_model
# from src.utils import plot_training_history # Uncomment if you put plotting functions in src.utils

# Specific Keras imports for callbacks (since they are instantiated directly here)
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Note: We no longer need to import Dense, GlobalAveragePooling2D, Input, ResNet50,
#       Adam, BinaryCrossentropy, BinaryAccuracy, Precision, Recall, l2 directly here,
#       because they are now imported and used *inside* the functions in src/model.py.

# For calculating class weights (still needed directly here for compute_class_weight)
from sklearn.utils import class_weight

print(f"TensorFlow version: {tf.__version__}")
print("Libraries imported successfully (using src modules).")

TensorFlow version: 2.19.0
Libraries imported successfully (using src modules).


In [2]:
# --- 2. Define Data Paths
base_dir = 'M:\Downloads\Pneumonia_Detection_XRay\Pneumonia_Detection_XRay\data\chest_xray'  

train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

# Verify that the directories exist
print("\n--- Verifying Data Directory Paths ---")
print(f"Train directory exists: {os.path.exists(train_dir)}")
print(f"Validation directory exists: {os.path.exists(val_dir)}")
print(f"Test directory exists: {os.path.exists(test_dir)}")
print("-" * 40)


--- Verifying Data Directory Paths ---
Train directory exists: True
Validation directory exists: True
Test directory exists: True
----------------------------------------


In [3]:
# --- 3. Data Preprocessing and Augmentation ---

# Define Image Size - ResNet50 typically expects 224x224
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32 # Common batch size, adjust based on GPU memory
NUM_CLASSES = 1 # Binary classification (Pneumonia or Normal)

# ResNet50's preprocess_input function
# This function rescales input pixels to [-1, 1], which is common for models pre-trained on ImageNet
preprocess_input = tf.keras.applications.resnet50.preprocess_input # <--- Make sure tf is imported!

print("\n--- Setting up Data Generators using src.data_processing ---")
train_generator, validation_generator, test_generator = get_image_data_generators(
    train_dir, val_dir, test_dir, IMG_HEIGHT, IMG_WIDTH, BATCH_SIZE, preprocess_input
)
print("-" * 40)


--- Setting up Data Generators using src.data_processing ---
Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
----------------------------------------


In [4]:
# --- 3.1. Handle Class Imbalance with Class Weights ---
print("\n--- Computing Class Weights using src.data_processing ---")
class_weights_dict = compute_class_weights(train_generator)

print(f"\nComputed Class Weights (for training imbalance): {class_weights_dict}")
print("This means misclassifying the minority class will be penalized more during training.")
print("-" * 40)


--- Computing Class Weights using src.data_processing ---

Computed Class Weights (for training imbalance): {0: 1.9448173005219984, 1: 0.6730322580645162}
This means misclassifying the minority class will be penalized more during training.
----------------------------------------


In [5]:
# --- 4 & 5. Load Pre-trained Model & Add Custom Classification Head (using src.model) ---
# Define regularization parameters (consistent with src/model.py defaults or your tuning)
DROPOUT_RATE = 0.0 # Start with 0.0 (no dropout) for initial testing if you want, then tune. Recommended: 0.4
L2_STRENGTH = 0.0 # Start with 0.0 (no L2) for initial testing if you want, then tune. Recommended: 0.001

print(f"\n--- Building Model with ResNet50 Base and Custom Head (Dropout: {DROPOUT_RATE}, L2: {L2_STRENGTH}) ---")

# This single line calls the function from src/model.py which defines the inputs,
# connects base_model, adds pooling, dense layers, dropout, and the output layer.
# It returns the complete 'model' and the 'base_model' instance.
model, base_model = build_transfer_model(IMG_HEIGHT, IMG_WIDTH, NUM_CLASSES,
                                         dropout_rate=DROPOUT_RATE, l2_strength=L2_STRENGTH)

model.summary() # Print summary here
print("-" * 40)


--- Building Model with ResNet50 Base and Custom Head (Dropout: 0.0, L2: 0.0) ---


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,112,513 (91.98 MB)

 Trainable params: 524,801 (2.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

----------------------------------------


In [6]:
# --- 6. Compile the Model (Phase 1: Frozen Layers) (using src.model) ---
print("\n--- Compiling Model (Phase 1: Frozen Layers) ---")
initial_learning_rate = 1e-4 # Define your learning rate here

model = compile_model(model, initial_learning_rate)
print("Model compiled for Phase 1 (Frozen Layers).")
print("-" * 40)


--- Compiling Model (Phase 1: Frozen Layers) ---
Model compiled for Phase 1 (Frozen Layers).
----------------------------------------


In [7]:
# --- 7. Train the Model (Phase 1: Frozen Layers) --- 
print("\n--- Starting Training (Phase 1: Frozen Layers) ---")

# Define callbacks
model_checkpoint_callback_phase1 = ModelCheckpoint(
    filepath=os.path.join('../models', 'best_model_phase1.h5'), # Save model to models/ folder
    monitor='val_accuracy', # Monitor validation accuracy
    save_best_only=True,    # Save only the best model
    mode='max',             # Maximize validation accuracy
    verbose=1
)

early_stopping_callback_phase1 = EarlyStopping(
    monitor='val_loss', # Monitor validation loss
    patience=5,         # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True, # Restore model weights from the epoch with the best value of the monitored quantity.
    verbose=1
)

# Number of epochs for initial training
EPOCHS_PHASE1 = 10 # Start with a reasonable number, early stopping will prevent overfitting

history_phase1 = model.fit(
    train_generator,
    epochs=EPOCHS_PHASE1,
    validation_data=validation_generator,
    class_weight=class_weights_dict, # Apply class weights to handle imbalance
    callbacks=[model_checkpoint_callback_phase1, early_stopping_callback_phase1],
    verbose=1
)

print("\nPhase 1 Training Complete. Best model saved to 'best_model_phase1.h5'.")
print("-" * 40)



--- Starting Training (Phase 1: Frozen Layers) ---


C:\Users\Melwin\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8354 - loss: 0.3213 - precision: 0.9424 - recall: 0.8309
Epoch 1: val_accuracy improved from -inf to 0.93750, saving model to ../models\best_model_phase1.h5


163/163 ━━━━━━━━━━━━━━━━━━━━ 344s 2s/step - accuracy: 0.8358 - loss: 0.3206 - precision: 0.9426 - recall: 0.8314 - val_accuracy: 0.9375 - val_loss: 0.2141 - val_precision: 1.0000 - val_recall: 0.8750
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9502 - loss: 0.1241 - precision: 0.9900 - recall: 0.9417
Epoch 2: val_accuracy did not improve from 0.93750
163/163 ━━━━━━━━━━━━━━━━━━━━ 309s 2s/step - accuracy: 0.9502 - loss: 0.1240 - precision: 0.9900 - recall: 0.9418 - val_accuracy: 0.9375 - val_loss: 0.1523 - val_precision: 0.8889 - val_recall: 1.0000
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9601 - loss: 0.0913 - precision: 0.9895 - recall: 0.9565
Epoch 3: val_accuracy did not improve from 0.93750
163/163 ━━━━━━━━━━━━━━━━━━━━ 294s 2s/step - accuracy: 0.9601 - loss: 0.0913 - precision: 0.9895 - recall: 0.9565 - val_accuracy: 0.8750 - val_loss: 0.1483 - val_precision: 0.8000 - val_recall: 1.0000
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - a

163/163 ━━━━━━━━━━━━━━━━━━━━ 282s 2s/step - accuracy: 0.9602 - loss: 0.1035 - precision: 0.9893 - recall: 0.9575 - val_accuracy: 1.0000 - val_loss: 0.0831 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9683 - loss: 0.0818 - precision: 0.9908 - recall: 0.9661
Epoch 5: val_accuracy did not improve from 1.00000
163/163 ━━━━━━━━━━━━━━━━━━━━ 281s 2s/step - accuracy: 0.9683 - loss: 0.0818 - precision: 0.9908 - recall: 0.9661 - val_accuracy: 1.0000 - val_loss: 0.0688 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9713 - loss: 0.0670 - precision: 0.9931 - recall: 0.9680
Epoch 6: val_accuracy did not improve from 1.00000
163/163 ━━━━━━━━━━━━━━━━━━━━ 282s 2s/step - accuracy: 0.9713 - loss: 0.0671 - precision: 0.9931 - recall: 0.9680 - val_accuracy: 1.0000 - val_loss: 0.0544 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - a

In [8]:
# --- 8. Fine-tuning (Phase 2: Unfrozen Layers) ---
print("\n--- Starting Fine-tuning (Phase 2: Unfrozen Layers) ---")

# Load the best weights from Phase 1 before fine-tuning
model.load_weights(os.path.join('../models', 'best_model_phase1.h5'))

# ... (model.load_weights line) ...

# Define fine-tuning parameters
NUM_UNFREEZE_LAYERS = 20 # Tune this
fine_tune_learning_rate = 1e-5 # Tune this

print(f"\n--- Unfreezing and Recompiling Model for Phase 2 (Fine-tuning) ---")
model = unfreeze_and_recompile_model(model, base_model, fine_tune_learning_rate, NUM_UNFREEZE_LAYERS)
model.summary() # See which layers are now trainable
print("-" * 40)

# Define callbacks for Phase 2
model_checkpoint_callback_phase2 = ModelCheckpoint(
    filepath=os.path.join('../models', 'final_best_model.h5'), # Save final best model
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

early_stopping_callback_phase2 = EarlyStopping(
    monitor='val_loss',
    patience=8, # Slightly more patience for fine-tuning
    restore_best_weights=True,
    verbose=1
)

reduce_lr_on_plateau = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2, # Reduce learning rate by a factor of 0.2
    patience=3, # If val_loss doesn't improve for 3 epochs, reduce LR
    min_lr=1e-7, # Minimum learning rate
    verbose=1
)

EPOCHS_PHASE2 = 30 # More epochs for fine-tuning, early stopping will manage it

history_phase2 = model.fit(
    train_generator,
    epochs=EPOCHS_PHASE2,
    validation_data=validation_generator,
    class_weight=class_weights_dict, # Continue applying class weights
    callbacks=[
        model_checkpoint_callback_phase2,
        early_stopping_callback_phase2,
        reduce_lr_on_plateau
    ],
    verbose=1
)

print("\nPhase 2 Fine-tuning Complete. Best model saved to 'final_best_model.h5'.")
print("-" * 40)


--- Starting Fine-tuning (Phase 2: Unfrozen Layers) ---

--- Unfreezing and Recompiling Model for Phase 2 (Fine-tuning) ---


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,112,513 (91.98 MB)

 Trainable params: 9,443,841 (36.03 MB)

 Non-trainable params: 14,668,672 (55.96 MB)

----------------------------------------
Epoch 1/30
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9659 - loss: 0.0815 - precision: 0.9897 - recall: 0.9645
Epoch 1: val_accuracy improved from -inf to 1.00000, saving model to ../models\final_best_model.h5


163/163 ━━━━━━━━━━━━━━━━━━━━ 303s 2s/step - accuracy: 0.9659 - loss: 0.0815 - precision: 0.9897 - recall: 0.9645 - val_accuracy: 1.0000 - val_loss: 0.0611 - val_precision: 1.0000 - val_recall: 1.0000 - learning_rate: 1.0000e-05
Epoch 2/30
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9694 - loss: 0.0689 - precision: 0.9950 - recall: 0.9640
Epoch 2: val_accuracy did not improve from 1.00000
163/163 ━━━━━━━━━━━━━━━━━━━━ 289s 2s/step - accuracy: 0.9694 - loss: 0.0689 - precision: 0.9950 - recall: 0.9640 - val_accuracy: 1.0000 - val_loss: 0.0757 - val_precision: 1.0000 - val_recall: 1.0000 - learning_rate: 1.0000e-05
Epoch 3/30
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9765 - loss: 0.0541 - precision: 0.9955 - recall: 0.9730
Epoch 3: val_accuracy did not improve from 1.00000
163/163 ━━━━━━━━━━━━━━━━━━━━ 289s 2s/step - accuracy: 0.9765 - loss: 0.0542 - precision: 0.9955 - recall: 0.9730 - val_accuracy: 1.0000 - val_loss: 0.0671 - val_precision: 1.0000 - val_recall: 1.000

In [9]:
# --- 9. Save Training History ---
# Combine history from both phases (if applicable) and save
history_df_phase1 = pd.DataFrame(history_phase1.history)
history_df_phase2 = pd.DataFrame(history_phase2.history)

# Adjust epoch numbers for phase 2 to be continuous
history_df_phase2.index = history_df_phase2.index + len(history_df_phase1)

# Concatenate histories
full_history_df = pd.concat([history_df_phase1, history_df_phase2])

history_save_path = os.path.join('../models', 'training_history.csv')
full_history_df.to_csv(history_save_path, index=False)
print(f"\nTraining history saved to {history_save_path}")

# --- 10. Final Model Saving (Redundant if checkpointing works, but good practice) ---
# If you want to ensure the final state of the model (after all callbacks) is saved
final_model_path = os.path.join('../models', 'trained_pneumonia_detector.h5')
model.save(final_model_path)
print(f"Final model (last state) saved to {final_model_path}")

print("\n--- Training Notebook Execution Complete ---")
print("You can now proceed to `3_model_evaluation.ipynb` to evaluate your best model.")


Training history saved to ../models\training_history.csv
Final model (last state) saved to ../models\trained_pneumonia_detector.h5

--- Training Notebook Execution Complete ---
You can now proceed to `3_model_evaluation.ipynb` to evaluate your best model.
